In [6]:
## loading dataset & environment
## https://pbpython.com/market-basket-analysis.html
## https://intellipaat.com/blog/data-science-apriori-algorithm/#What-Is-Association-Rule-Mining
## https://stackabuse.com/association-rule-mining-via-apriori-algorithm-in-python/
## https://www.edureka.co/blog/apriori-algorithm/
## http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/

import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
df.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [0]:
# Stripping extra spaces in the description 
df['Description'] = df['Description'].str.strip() 
  
# Dropping the rows without any invoice number 
df.dropna(axis = 0, subset =['InvoiceNo'], inplace = True) 
df['InvoiceNo'] = df['InvoiceNo'].astype('str') 
  
# Dropping all transactions which were done on credit 
df = df[~df['InvoiceNo'].str.contains('C')] 

In [9]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [0]:
basket_Italy = (df[df['Country'] =="Italy"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 

In [11]:
basket_Italy.head()

Description,12 EGG HOUSE PAINTED WOOD,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,16 PIECE CUTLERY SET PANTRY DESIGN,20 DOLLY PEGS RETROSPOT,3 GARDENIA MORRIS BOXED CANDLES,3 ROSE MORRIS BOXED CANDLES,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN RED AND CREAM,3 TRADITIONAl BISCUIT CUTTERS SET,36 FOIL HEART CAKE CASES,5 HOOK HANGER RED MAGIC TOADSTOOL,6 CHOCOLATE LOVE HEART T-LIGHTS,6 EGG HOUSE PAINTED WOOD,6 GIFT TAGS VINTAGE CHRISTMAS,72 SWEETHEART FAIRY CAKE CASES,ABC TREASURE BOOK BOX,ADULT APRON APPLE DELIGHT,APRON APPLE DELIGHT,ASSORTED COLOUR BIRD ORNAMENT,ASSORTED COLOUR MINI CASES,ASSORTED COLOURS SILK FAN,BAG 125g SWIRLY MARBLES,BAKING MOULD CHOCOLATE CUPCAKES,BAKING SET 9 PIECE RETROSPOT,BAKING SET SPACEBOY DESIGN,BATH BUILDING BLOCK WORD,BEWARE OF THE CAT METAL SIGN,BIG DOUGHNUT FRIDGE MAGNETS,BINGO SET,BIRDS MOBILE VINTAGE DESIGN,BISCUIT TIN 50'S CHRISTMAS,BLACK/BLUE POLKADOT UMBRELLA,BLUE FLOCK GLASS CANDLEHOLDER,BLUE HAPPY BIRTHDAY BUNTING,BLUE POLKADOT WRAP,BOTTLE BAG RETROSPOT,BOUDOIR SQUARE TISSUE BOX,BOX OF 6 MINI 50'S CRACKERS,...,TRADITIONAL CHRISTMAS RIBBONS,TRADITIONAL KNITTING NANCY,TRADITIONAL WOODEN SKIPPING ROPE,TRADTIONAL ALPHABET STAMP SET,TREASURE ISLAND BOOK BOX,TV DINNER TRAY AIR HOSTESS,TV DINNER TRAY DOLLY GIRL,VICTORIAN GLASS HANGING T-LIGHT,VICTORIAN SEWING KIT,VINTAGE BILLBOARD LOVE/HATE MUG,VINTAGE CHRISTMAS GIFT SACK,VINTAGE CREAM CAT FOOD CONTAINER,VINTAGE CREAM DOG FOOD CONTAINER,VINTAGE DOILY DELUXE SEWING KIT,VINTAGE DOILY JUMBO BAG RED,VINTAGE DOILY TRAVEL SEWING KIT,VINTAGE UNION JACK CUSHION COVER,VINTAGE UNION JACK DOORSTOP,VINTAGE UNION JACK MEMOBOARD,VINTAGE UNION JACK SHOPPING BAG,WATERING CAN BLUE ELEPHANT,WATERING CAN GREEN DINOSAUR,WELCOME WOODEN BLOCK LETTERS,WHITE BAROQUE WALL CLOCK,WHITE HANGING HEART T-LIGHT HOLDER,WOOD BLACK BOARD ANT WHITE FINISH,WOOD S/3 CABINET ANT WHITE FINISH,WOODEN CROQUET GARDEN SET,WOODEN SCHOOL COLOURING SET,WOODLAND HEIGHT CHART STICKERS,WOODLAND BUNNIES LOLLY MAKERS,WOODLAND CHARLOTTE BAG,WRAP DOILEY DESIGN,WRAP ENGLISH ROSE,WRAP I LOVE LONDON,WRAP RED APPLES,WRAP RED VINTAGE DOILY,YOU'RE CONFUSING ME METAL SIGN,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
537022,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
539752,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
541115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
541703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
542238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
def hot_encode(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_encoded = basket_Italy.applymap(hot_encode) 
basket_Italy = basket_encoded 

In [16]:
basket_Italy.head()

Description,12 EGG HOUSE PAINTED WOOD,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,16 PIECE CUTLERY SET PANTRY DESIGN,20 DOLLY PEGS RETROSPOT,3 GARDENIA MORRIS BOXED CANDLES,3 ROSE MORRIS BOXED CANDLES,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN RED AND CREAM,3 TRADITIONAl BISCUIT CUTTERS SET,36 FOIL HEART CAKE CASES,5 HOOK HANGER RED MAGIC TOADSTOOL,6 CHOCOLATE LOVE HEART T-LIGHTS,6 EGG HOUSE PAINTED WOOD,6 GIFT TAGS VINTAGE CHRISTMAS,72 SWEETHEART FAIRY CAKE CASES,ABC TREASURE BOOK BOX,ADULT APRON APPLE DELIGHT,APRON APPLE DELIGHT,ASSORTED COLOUR BIRD ORNAMENT,ASSORTED COLOUR MINI CASES,ASSORTED COLOURS SILK FAN,BAG 125g SWIRLY MARBLES,BAKING MOULD CHOCOLATE CUPCAKES,BAKING SET 9 PIECE RETROSPOT,BAKING SET SPACEBOY DESIGN,BATH BUILDING BLOCK WORD,BEWARE OF THE CAT METAL SIGN,BIG DOUGHNUT FRIDGE MAGNETS,BINGO SET,BIRDS MOBILE VINTAGE DESIGN,BISCUIT TIN 50'S CHRISTMAS,BLACK/BLUE POLKADOT UMBRELLA,BLUE FLOCK GLASS CANDLEHOLDER,BLUE HAPPY BIRTHDAY BUNTING,BLUE POLKADOT WRAP,BOTTLE BAG RETROSPOT,BOUDOIR SQUARE TISSUE BOX,BOX OF 6 MINI 50'S CRACKERS,...,TRADITIONAL CHRISTMAS RIBBONS,TRADITIONAL KNITTING NANCY,TRADITIONAL WOODEN SKIPPING ROPE,TRADTIONAL ALPHABET STAMP SET,TREASURE ISLAND BOOK BOX,TV DINNER TRAY AIR HOSTESS,TV DINNER TRAY DOLLY GIRL,VICTORIAN GLASS HANGING T-LIGHT,VICTORIAN SEWING KIT,VINTAGE BILLBOARD LOVE/HATE MUG,VINTAGE CHRISTMAS GIFT SACK,VINTAGE CREAM CAT FOOD CONTAINER,VINTAGE CREAM DOG FOOD CONTAINER,VINTAGE DOILY DELUXE SEWING KIT,VINTAGE DOILY JUMBO BAG RED,VINTAGE DOILY TRAVEL SEWING KIT,VINTAGE UNION JACK CUSHION COVER,VINTAGE UNION JACK DOORSTOP,VINTAGE UNION JACK MEMOBOARD,VINTAGE UNION JACK SHOPPING BAG,WATERING CAN BLUE ELEPHANT,WATERING CAN GREEN DINOSAUR,WELCOME WOODEN BLOCK LETTERS,WHITE BAROQUE WALL CLOCK,WHITE HANGING HEART T-LIGHT HOLDER,WOOD BLACK BOARD ANT WHITE FINISH,WOOD S/3 CABINET ANT WHITE FINISH,WOODEN CROQUET GARDEN SET,WOODEN SCHOOL COLOURING SET,WOODLAND HEIGHT CHART STICKERS,WOODLAND BUNNIES LOLLY MAKERS,WOODLAND CHARLOTTE BAG,WRAP DOILEY DESIGN,WRAP ENGLISH ROSE,WRAP I LOVE LONDON,WRAP RED APPLES,WRAP RED VINTAGE DOILY,YOU'RE CONFUSING ME METAL SIGN,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
537022,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
539752,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
541115,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
541703,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
542238,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
frq_items = apriori(basket_Italy, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

In [11]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
22,(RED LOVE HEART SHAPE CUP),(BAKING MOULD CHOCOLATE CUPCAKES),0.052632,0.052632,0.052632,1.000000,19.000000,0.049861,inf
23,(BAKING MOULD CHOCOLATE CUPCAKES),(RED LOVE HEART SHAPE CUP),0.052632,0.052632,0.052632,1.000000,19.000000,0.049861,inf
54,(BATH BUILDING BLOCK WORD),(HOME BUILDING BLOCK WORD),0.052632,0.052632,0.052632,1.000000,19.000000,0.049861,inf
55,(HOME BUILDING BLOCK WORD),(BATH BUILDING BLOCK WORD),0.052632,0.052632,0.052632,1.000000,19.000000,0.049861,inf
164,(CHRISTMAS CRAFT WHITE FAIRY),(CHRISTMAS CRAFT LITTLE FRIENDS),0.052632,0.052632,0.052632,1.000000,19.000000,0.049861,inf
...,...,...,...,...,...,...,...,...,...
4009,(POSTAGE),"(SET OF 3 CAKE TINS PANTRY DESIGN, DOORMAT WEL...",0.447368,0.105263,0.052632,0.117647,1.117647,0.005540,1.014035
8420,(POSTAGE),"(SET OF 20 KIDS COOKIE CUTTERS, PLASTERS IN TI...",0.447368,0.105263,0.052632,0.117647,1.117647,0.005540,1.014035
8725,(POSTAGE),"(RECYCLING BAG RETROSPOT, TOY TIDY PINK POLKADOT)",0.447368,0.105263,0.052632,0.117647,1.117647,0.005540,1.014035
8731,(POSTAGE),"(RECYCLING BAG RETROSPOT, TOY TIDY SPACEBOY)",0.447368,0.105263,0.052632,0.117647,1.117647,0.005540,1.014035


In [19]:
rules[ (rules['lift'] >= 3) &
       (rules['confidence'] >= 0.3) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
22,(RED LOVE HEART SHAPE CUP),(BAKING MOULD CHOCOLATE CUPCAKES),0.052632,0.052632,0.052632,1.000000,19.000000,0.049861,inf
23,(BAKING MOULD CHOCOLATE CUPCAKES),(RED LOVE HEART SHAPE CUP),0.052632,0.052632,0.052632,1.000000,19.000000,0.049861,inf
54,(BATH BUILDING BLOCK WORD),(HOME BUILDING BLOCK WORD),0.052632,0.052632,0.052632,1.000000,19.000000,0.049861,inf
55,(HOME BUILDING BLOCK WORD),(BATH BUILDING BLOCK WORD),0.052632,0.052632,0.052632,1.000000,19.000000,0.049861,inf
164,(CHRISTMAS CRAFT LITTLE FRIENDS),(CHRISTMAS CRAFT WHITE FAIRY),0.052632,0.052632,0.052632,1.000000,19.000000,0.049861,inf
...,...,...,...,...,...,...,...,...,...
10378,(BREAD BIN DINER STYLE IVORY),"(DOORMAT AIRMAIL, DOORMAT WELCOME TO OUR HOME,...",0.157895,0.105263,0.052632,0.333333,3.166667,0.036011,1.342105
16662,(PLASTERS IN TIN CIRCUS PARADE),"(CHILDRENS APRON APPLES DESIGN, TOY TIDY PINK ...",0.157895,0.105263,0.052632,0.333333,3.166667,0.036011,1.342105
17054,(ROUND SNACK BOXES SET OF4 WOODLAND),"(CHILDRENS APRON APPLES DESIGN, TOY TIDY PINK ...",0.157895,0.105263,0.052632,0.333333,3.166667,0.036011,1.342105
19870,(SET OF 3 CAKE TINS PANTRY DESIGN),"(DOORMAT AIRMAIL, DOORMAT WELCOME TO OUR HOME,...",0.236842,0.105263,0.078947,0.333333,3.166667,0.054017,1.342105


In [0]:
basket['ALARM CLOCK BAKELIKE GREEN'].sum()

340.0

In [0]:
basket['ALARM CLOCK BAKELIKE RED'].sum()

316.0

In [0]:
basket2 = (df[df['Country'] =="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.05, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)

rules2[ (rules2['lift'] >= 4) &
        (rules2['confidence'] >= 0.5)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.115974,0.137856,0.067834,0.584906,4.242887,0.051846,2.076984
7,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN WOODLAND ANIMALS),0.107221,0.137856,0.061269,0.571429,4.145125,0.046488,2.011670
11,(RED RETROSPOT CHARLOTTE BAG),(WOODLAND CHARLOTTE BAG),0.070022,0.126915,0.059081,0.843750,6.648168,0.050194,5.587746
